In [44]:
# in the previous code the elements were rectangles/quadrilaterals
# -> in this code they are BARS (LINES) = simpler!
# useful reference by Mert Solen (https://youtu.be/tv1TlAebvm0)

import numpy as np

# auxiliary ftions: extended_nodes updates
# returns local DOF matrix, global DOF matrix, tot num of DOC, tot num of DOF
def get_DOF(nodes, BC):
    num_nodes, PD = len(nodes), len(nodes[0])

    DOF = 0  # degrees of freedom
    DOC = 0  # degrees of constraint

    local_DOF = np.zeros((num_nodes, PD))
    global_DOF = np.zeros((num_nodes, PD))

    for i in range(num_nodes):
        for j in range(PD):
            if BC[i, j] == -1:
                DOC -= 1
                local_DOF[i, j] = DOC
            else:
                DOF += 1
                local_DOF[i, j] = DOF

    for i in range(num_nodes):
        for j in range(PD):
            if local_DOF[i, j] < 0:
                global_DOF[i, j] = abs(local_DOF[i, j]) + DOF
            else:
                global_DOF[i, j] = abs(local_DOF[i, j])

    return (local_DOF, global_DOF, DOC, DOF)

# obtain the element stiffness matrix based on the list of nodes in an element (el_list)
def get_element_stiffness(el_list, ext_nodes, E, A):
    X1 = ext_nodes[el_list[0], 0]
    Y1 = ext_nodes[el_list[0], 1]
    X2 = ext_nodes[el_list[1], 0]
    Y2 = ext_nodes[el_list[1], 1]

    L = np.sqrt((X1 - X2)**2 + (Y1 - Y2)**2)
    c = (X2 - X1) / L
    s = (Y2 - Y1) / L
    KE = E*A/L * np.array([[c**2, c * s, -c**2, -c * s],
                           [c * s, s**2, - c * s, -s**2],
                           [-c**2, -c * s, c**2, c * s],
                           [-c * s, -s**2, s * c, s**2]])
    return KE

# assemble the stiffness matrix K
def get_global_stiffness(nodes, conn, ext_nodes, E, A):
    num_nodes, num_el, PD = len(nodes), len(conn), len(nodes[0])
    K = np.zeros((num_nodes * PD, num_nodes * PD))  # init (global) stiffness matrix
    L = [np.sqrt(nodes[i,0]**2 + nodes[i,1]**2) for i in range(len(conn))]
    for i in range(num_el):  # obtain individual element stiffnesses
        element_list = conn[i,:]
        KE = get_element_stiffness(element_list, ext_nodes, E, A)
        for k in range(len(element_list)):  # for each node within element
            for l in range(PD):             # get each coordinate
                for m in range(len(element_list)):
                    for n in range(PD):
                        row = ext_nodes[element_list[k], l + 3*PD]
                        col = ext_nodes[element_list[m], n + 3*PD]
                        val = KE[k*PD + l, m*PD + n]

                        K[int(row) - 1, int(col) - 1] += val
    return K

# assemble the unknown forces
def get_forces(nodes, ext_nodes):
    num_nodes, PD = len(nodes), len(nodes[0])
    DOF = 0
    F_new = []

    for i in range(num_nodes):  # for each node
        for j in range(PD):     # for each dimension
            if ext_nodes[i, PD + j] == 1: # if node is not fixed
                DOF += 1
                F_new.append(ext_nodes[i, 5*PD+j])
    return np.array(F_new)

# assemble the unknown displacements
def get_displacements(nodes, ext_nodes):
    num_nodes, PD = len(nodes), len(nodes[0])
    DOC = 0
    U_new = []

    for i in range(num_nodes):  # for each node
        for j in range(PD):     # for each dimension
            if ext_nodes[i, PD + j] == -1: # if node is not fixed
                DOC += 1
                U_new.append(ext_nodes[i, 4*PD+j])
    return np.array(U_new)

# updates the extended nodes with the solution values for displacements and forces
def update_ext_nodes(nodes, ext_nodes, U_new, F_u):
    num_nodes, PD = len(nodes), len(nodes[0])

    DOF = 0  # degrees of freedom
    DOC = 0  # degrees of constraint

    for i in range(num_nodes):
        for j in range(PD):
            if ext_nodes[i, PD+j] == 1:
                DOF += 1
                ext_nodes[i, 4*PD + j] = U_new[DOF - 1]
            else:
                DOC += 1
                ext_nodes[i, 5*PD + j] = F_u[DOC - 1]
    return ext_nodes


In [45]:
# material specifications
E = 10.0 ** 6   # Youngs modulus
A = 0.01        # cross-section area of a bar

# defining the structure
nodes = np.array([[0, 0],
                  [1, 0],
                  [0.5, 1]])
conn = np.array([[0, 1],
                 [1, 2],
                 [2, 0]])
num_nodes = len(nodes)  # total number of nodes
PD = 2                  # problem dimension: 2D

# assigning boundary conditions
BC = np.array([[-1, -1],  # Dirichlet & Neumann Boundary Conditions
               [1, -1],
               [1, 1]])
    # (-1,-1) -> fixed in X and Y
    # ( 1,-1) -> free to move in X and fixed in Y

# assigning nodal forces
F = np.array([[0,  0],
              [0,  0],
              [0, -20]])

# initial displacements
U = np.array([[0, 0],
              [0, 0],
              [0, 0]])

# extended node list (ENL)
(localDOF, globalDOF, DOC, DOF) = get_DOF(nodes=nodes, BC=BC)  # get the DOF matrices
ext_nodes = np.zeros((num_nodes, 6 * PD))  # nodes, BC, local DoF, global DoF, Displacement, Forces
ext_nodes[:, 0:PD] = nodes          # first 2 columns are node list
ext_nodes[:, PD:2*PD] = BC          # boundary conditions
ext_nodes[:, 2*PD:3*PD] = localDOF  # local DOFs
ext_nodes[:, 3*PD:4*PD] = globalDOF # global DOFs
ext_nodes[:, 4*PD:5*PD] = U         # displacements
ext_nodes[:, 5*PD:6*PD] = F         # forces

K = get_global_stiffness(nodes=nodes, conn=conn, ext_nodes=ext_nodes, E=E, A=A)  # calculating global stiffness
U, F = U.flatten(), F.flatten()

# finding the new displacements and forces
F_p = get_forces(nodes, ext_nodes)        # force prescribed
U_p = get_displacements(nodes, ext_nodes) # displacement prescribed

# using F = K * U
K_UU = K[0:DOF, 0:DOF]
K_UP = K[0:DOF, DOF:DOF+DOC]
K_PU = K[DOF:DOF+DOC, 0:DOF]
K_PP = K[DOF:DOF+DOC, DOF:DOF+DOC]

F_new = F_p - np.matmul(K_UP, U_p)
U_new = np.matmul(np.linalg.inv(K_UU), F_new)
F_u = np.matmul(K_PU, U_new) + np.matmul(K_PP, U_p)


print(ext_nodes)



AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [14]:
a0

array([1, 2, 3, 4, 5])

In [15]:
a1

array([2, 3, 4, 5, 6])